In [29]:
import pandas as pd
import numpy as np
import pickle
import math

In [30]:
file_to_read = open("user2movie.json", "rb")
user2movies = pickle.load(file_to_read)

file_to_read = open("movie2user.json", "rb")
movie2users = pickle.load(file_to_read)

file_to_read = open("usermovie2rating.json", "rb")
usermovie2rating = pickle.load(file_to_read)

file_to_read = open("usermovie2rating_test.json", "rb")
usermovie2rating_test = pickle.load(file_to_read)

In [31]:
def user_avg_rating(user_id):
    count_rating =0
    sum_rating = 0
    for movies in user2movies[user_id]:#iterating over all user id rated movies
        sum_rating += usermovie2rating[(user_id, movies)]#sum the rating
        count_rating+=1
    avg_rating = sum_rating/count_rating
    return avg_rating

    

In [32]:
def calc_single_w(user_a, user_b):
#getting two user's id, returning the weight by peerson
    user_a_mean = user_avg_rating(user_a)#R_a- can shorten...calc avg for all user in advanced
    user_b_mean = user_avg_rating(user_b)#R_b
    numerator = 0
    denominator = 0
    dom_a = 0
    dom_b = 0
    #finding the subset of intersection between movies rated by a and b
    set_a = user2movies[user_a]
    set_b = user2movies[user_b]
    intersection = set(set(set_a) & set(set_b))
    if len(intersection)<5:#if intersection to low, ignore this user
        return 0
    for movie in intersection:
        movie_rating_a = usermovie2rating[(user_a, movie)]
        movie_rating_b = usermovie2rating[(user_b, movie)]
        movie_dev_a = movie_rating_a - user_a_mean#user a rating deviation of movie
        movie_dev_b = movie_rating_b - user_b_mean#user b rating deviation of movie
        numerator += movie_dev_a*movie_dev_b
        dom_a += movie_dev_a**2
        dom_b += movie_dev_b**2
    dom_a = math.sqrt(dom_a)
    dom_b = math.sqrt(dom_b)
    denominator = dom_a*dom_b
    w = numerator/denominator
    return w
    

In [33]:

def calculate_all_w():
    #calculating all the weights, putting into dict
    user_user_w = {}
    k=25
    for user_i in user2movies.keys():#running over all users
        for user_j in user2movies.keys():
            if user_i == user_j:
                continue
            if (user_j, user_i) in user_user_w:
                user_user_w[(user_i, user_j)] = user_user_w[(user_j, user_i)]#correlation is symetric so if already in dict no need to calc again
            else:
                 user_user_w[(user_i, user_j)] = calc_single_w(user_i, user_j)
        
    return user_user_w



In [54]:
from heapq import nlargest
def neighborfy(user_id, user_user_w):
#should get a subset of users who rated the movie to k top user's weights to our user
   
    k=40
    userid_user_w = {}
    for user in user2movies.keys():#looping throw all users
        if user == user_id:
            continue
        else:
            userid_user_w[user] = abs(user_user_w[user_id, user])#taking the two users weight to the new user_id dict
    
    neighbors = nlargest(k, userid_user_w, key = userid_user_w.get)#geeting the k best neighbors
    return neighbors



In [35]:
def get_score(user_id, movie_id, user_user_w, neighborhood, user_avg):

    user_rated_item_full= movie2users[movie_id]# the subset of users who rated the item movie_id 
    user_rated_item = set(set(user_rated_item_full) & set(neighborhood))#the intersection from the userid neighbours
    w = {}# a dict containing all the wheights between user_id and other users from his neighbors who rated the item
    userId_mean = user_avg[user_id]#user_id avg rating of all movies
    #TO DO- calc in advance the rating- not depended on movie!
    numerator = 0
    denominator = 0
    for user in user_rated_item:
        w[user]= user_user_w[(user_id, user)]# fetch current user and user_id weight. 
        user_rating= user_avg[user]#calculate current user avg
        deviation = user_rating - userId_mean
        numerator+= w[user]*deviation
        denominator = denominator+abs(w[user])
    
    
    if denominator == 0:
        return -10000  
    score = userId_mean+ (numerator/denominator)
    return score
        


In [36]:
def calc_mse(test, pred):
    error =0
    count = 0
    for pair in test.keys():
        error += (test[pair]-pred[pair])**2
        count+=1
    mse = error/count
    return mse


In [37]:
#this is the call for w calculation- it takes 175 min. so we ran it once and saved as file.
#w = calculate_all_w()#calculate w on train dicts
#with open('weights.json', 'wb') as f:
#  pickle.dump(w, f)

In [38]:
file_to_read = open("weights.json", "rb")
w = pickle.load(file_to_read)

In [39]:
len(w)

24995000

In [55]:
#calc all users avg and neighbors
user_avg = {}
neighborhood = {}
for user in user2movies.keys():
    neighborhood[user] = neighborfy(user, w)#lets find his neiborhood
    user_avg[user] = user_avg_rating(user)


In [56]:

scores = {}
i=0
errors = []
for pair in usermovie2rating_test.keys():
    user = pair[0]   
    scores[pair] = get_score(pair[0], pair[1], w, neighborhood[pair[0]], user_avg)#for every pair in the test lets find the score
    if scores[pair] == -10000:
        scores.pop(pair)
        errors.append(pair)
        print("error calculating %s prediction" %(pair,))
    i+=1


In [57]:
errors

[]

In [58]:
def error_remove():
    for error in errors:
        usermovie2rating_test.pop(error)
    MSE = calc_mse(scores, usermovie2rating_test)
    print("mse:", MSE)
    pass

In [61]:
len(errors)
#great!

0

In [60]:
for error in errors:
    usermovie2rating_test.pop(error)
MSE = calc_mse(scores, usermovie2rating_test)
print("mse:", MSE)
    

mse: 0.9765449056416119
